In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
%pip install langchain_openai==0.3.7

  Using cached langchain_openai-0.3.7-py3-none-any.whl.metadata (2.3 kB)
  Using cached tiktoken-0.9.0-cp311-cp311-win_amd64.whl.metadata (6.8 kB)
Using cached langchain_openai-0.3.7-py3-none-any.whl (55 kB)
   ---------------------------------------- 0.0/755.1 kB ? eta -:--:--
   ---------- ----------------------------- 194.6/755.1 kB 5.9 MB/s eta 0:00:01
   ----------------------------------- --- 686.1/755.1 kB 10.9 MB/s eta 0:00:01
   ---------------------------------------- 755.1/755.1 kB 9.5 MB/s eta 0:00:00
Using cached tiktoken-0.9.0-cp311-cp311-win_amd64.whl (893 kB)
   ---------------------------------------- 0.0/209.2 kB ? eta -:--:--
   --------------------------------------- 209.2/209.2 kB 12.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
model_name = "BAAI/bge-m3"
hf_embeddings = HuggingFaceEmbeddings(model_name=model_name)

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=0,
    chunk_overlap=0
)

loader = TextLoader("./docs/travel.txt", encoding="utf-8")
documents = loader.load_and_split(text_splitter=text_splitter)

Created a chunk of size 52, which is longer than the specified 0
Created a chunk of size 46, which is longer than the specified 0
Created a chunk of size 43, which is longer than the specified 0
Created a chunk of size 47, which is longer than the specified 0
Created a chunk of size 41, which is longer than the specified 0
Created a chunk of size 41, which is longer than the specified 0
Created a chunk of size 47, which is longer than the specified 0
Created a chunk of size 38, which is longer than the specified 0
Created a chunk of size 40, which is longer than the specified 0
Created a chunk of size 41, which is longer than the specified 0
Created a chunk of size 38, which is longer than the specified 0
Created a chunk of size 46, which is longer than the specified 0
Created a chunk of size 36, which is longer than the specified 0
Created a chunk of size 35, which is longer than the specified 0
Created a chunk of size 45, which is longer than the specified 0
Created a chunk of size 3

In [7]:
from langchain_chroma import Chroma
db = Chroma.from_documents(documents, hf_embeddings)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [8]:
from langchain.retrievers.multi_query import MultiQueryRetriever
retriever = MultiQueryRetriever.from_llm(retriever=db.as_retriever(), llm=llm)

In [11]:
retriever.invoke("여행 계획 세우기")

[chain/start] [retriever:MultiQueryRetriever > chain:RunnableSequence] Entering Chain run with input:
{
  "question": "여행 계획 세우기"
}
[chain/start] [retriever:MultiQueryRetriever > chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "question": "여행 계획 세우기"
}
[chain/end] [retriever:MultiQueryRetriever > chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [retriever:MultiQueryRetriever > chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an AI language model assistant. Your task is \n    to generate 3 different versions of the given user \n    question to retrieve relevant documents from a vector  database. \n    By generating multiple perspectives on the user question, \n    your goal is to help the user overcome some of the limitations \n    of distance-based similarity search. Provide these alternative \n    questions separated by newlines. Origina

[Document(id='39f1fe7e-71ea-4118-a107-0cc2038c0c5e', metadata={'source': './docs/travel.txt'}, page_content='여행 일정 짜는 방법. 관광 명소를 효과적으로 방문하기 위한 스케줄링 팁.'),
 Document(id='2dfebecb-797d-4ccf-8b58-14f9b97f7a93', metadata={'source': './docs/travel.txt'}, page_content='효율적인 여행 계획 세우기. 예산을 절약하면서도 만족스러운 여행을 하는 방법.'),
 Document(id='cbec3d53-29df-4428-93fe-76641ee24c75', metadata={'source': './docs/travel.txt'}, page_content='여행 경로를 설계하는 법. 최적의 동선을 계획하여 피로도를 줄이는 방법을 소개합니다.'),
 Document(id='bbea7e7e-58f8-47f8-a269-d8054e83a512', metadata={'source': './docs/travel.txt'}, page_content='여행을 준비하는 방법은 무엇인가? 필요한 짐을 꾸리는 팁과 함께 여행의 기본 요소를 알아보자.'),
 Document(id='6b42150b-a27d-4ec0-afa9-3a6664ddbeb1', metadata={'source': './docs/travel.txt'}, page_content='여행 중 음식 고르는 법. 지역 특산물을 즐기면서도 위생을 고려하는 요령.')]

In [10]:
import langchain
langchain.debug = True